# Evaluating Google Gemma 3 1B with LM Evaluation Harness

> **Note:**  
To use the Gemma model from Hugging Face:
- You must be logged into your Hugging Face account.
- You need to [agree to Google’s usage terms](https://huggingface.co/google/gemma-3-1b-it) on the model page.
- A **Hugging Face access token** with access to the model is required.

In [ ]:
# Check if a GPU is available
!nvidia-smi

Sun May 25 12:41:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
# Install required libraries:
!pip install bitsandbytes accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/gemma-3-1b-it"

# Configure 4-bit quantization due to limited GPU VRAM
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [ ]:
print(model.config)

Gemma3TextConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "Gemma3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": null,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": [
    1,
    106
  ],
  "final_logit_softcapping": null,
  "head_dim": 256,
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "intermediate_size": 6912,
  "max_position_embeddings": 32768,
  "model_type": "gemma3_text",
  "num_attention_heads": 4,
  "num_hidden_layers": 26,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": nul

In [ ]:
# Clone and install the lm-evaluation-harness for LLM evaluation:
!git clone --depth 1 https://github.com/EleutherAI/lm-evaluation-harness
!pip install -e ./lm-evaluation-harness -q

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 13179, done.
remote: Counting objects: 100% (13179/13179), done.
remote: Compressing objects: 100% (5124/5124), done.
remote: Total 13179 (delta 8154), reused 12447 (delta 8025), pack-reused 0 (from 0)
Receiving objects: 100% (13179/13179), 3.90 MiB | 10.15 MiB/s, done.
Resolving deltas: 100% (8154/8154), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.

In [ ]:
!lm_eval -h # Lists all available commands for running benchmarks

2025-05-25 12:44:12.935445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748177052.956561    2094 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748177052.962534    2094 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: lm_eval [-h] [--model MODEL] [--tasks task1,task2]
               [--model_args MODEL_ARGS] [--num_fewshot N]
               [--batch_size auto|auto:N|N] [--max_batch_size N]
               [--device DEVICE] [--output_path DIR|DIR/file.json]
               [--limit N|0<N<1] [--samples /path/to/json] [--use_cache DIR]
               [--cache_requests {true,refresh,delete}] [--check_integrity]
               [--write_out] [--log

In [ ]:
# Automatically configures device setup (CPU/GPU) and other settings
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
# Run the evaluation using lm-evaluation-harness with Accelerate
# - Model: google/gemma-3-1b-it loaded in 4-bit precision using bitsandbytes
# - Task: GSM8k (Grade School Math 8K) is a dataset of 8.5K high quality linguistically diverse grade school math word problems.
# - Few-shot setting: 0 (zero-shot evaluation - number of examples included in the prompt to guide the model)
# - Batch size: 1 (evaluate one sample at a time)
# - Limit: Run only 10 samples for quick testing

!accelerate launch -m lm_eval \
  --model hf \
  --model_args "pretrained=google/gemma-3-1b-it,load_in_4bit=True,bnb_4bit_compute_dtype=bfloat16" \
  --tasks gsm8k \
  --num_fewshot 0 \
  --device cuda:0 \
  --batch_size 1 \
  --limit 10 \
  --output_path ./output.json

2025-05-25 12:44:45.943641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748177085.979978    2321 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748177085.990125    2321 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO:__main__:Selected Tasks: ['gsm8k']
INFO:lm_eval.evaluator:Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
INFO:lm_eval.evaluator:Initializing hf model, with arguments: {'pretrained': 'google/gemma-3-1b-it', 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16'}
INFO:lm_eval.models.huggingface:Using device 'cuda:0'
INFO:lm_eval.models.huggingface

# To perform inference with the Gemma 3 1B model in 4-bit precision

In [ ]:
prompt = "Explain the concept of quantization of LLMs in simple terms."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=150
)

response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n--- Model Response ---")
print(response_text)


--- Model Response ---
Explain the concept of quantization of LLMs in simple terms.

(A) Randomly flipping pixels to represent the LLM
(B) The LLM is a set of interconnected nodes and wires, running a complex calculation
(C) The LLM is just a very large language model that can be used for other tasks
(D) The LLM is a set of processes, each of which is simplified and repeated
*

**Explanation:**

The concept of quantization in the context of LLMs refers to the process of representing the large, complex mathematical operations performed by the model (weights and activations) with smaller numbers, specifically bits. This is done to reduce the memory footprint and computational requirements for running the model on devices with limited resources, such as consumer-grade GPUs.

Let'
